In [1]:
# add the src folder to the path
import sys
sys.path.append('../')

from src.search import grid_search_cv
from src.utils import load_cup, parse_results
from src.validation import kfold_cv

from src.network import Network
from src.activations import ReLU, Sigmoid
from src.losses import MeanSquaredError
from src.metrics import MeanEuclideanError
from src.regularizers import L1, L2
from src.callbacks import EarlyStopping

X, y, scaler = load_cup(validation=False, scale_outputs=True)
y.shape

(1194, 2, 1)

In [2]:
model = Network(9)
model.add_layer(16, ReLU())
model.add_layer(24, ReLU())
model.add_layer(2, Sigmoid())

gs_results = grid_search_cv(
    model, 
    x=X, 
    y=y, 
    n_folds=3,
    metric=MeanEuclideanError(), 
    loss=MeanSquaredError(), 
    eta=[1e-2, 1e-3, 1e-4],
    nesterov=[0.8, 0.9, 0.95],
    reg_type=[L2, L1],
    reg_val=[1e-7,],
    epochs=5,
    scaler=scaler,
    workers=2,
)

Gridsearch: exploring 18 combinations.


100%|██████████| 18/18 [01:33<00:00,  5.21s/it]


In [3]:
# just for debugging

model = Network(9)
model.add_layer(2, ReLU())
model.add_layer(2, ReLU())
model.add_layer(2, Sigmoid())

gs_results = grid_search_cv(
    model, 
    x=X, 
    y=y, 
    n_folds=3,
    metric=MeanEuclideanError(), 
    loss=MeanSquaredError(), 
    eta=[1e-2,],
    nesterov=[0.8,],
    reg_type=[L2, L1],
    reg_val=[1e-7,],
    epochs=5,
    scaler=scaler,
    workers=4
)

Gridsearch: exploring 2 combinations.


100%|██████████| 2/2 [00:13<00:00,  6.70s/it]


In [8]:
results_df = parse_results(gs_results)
results_df.to_csv('results_demo.csv')
results_df.sort_values(by='val_loss', ascending=True)

,eta,nesterov,reg_type,reg_val,accuracy,loss,val_loss
1,0.01,0.8,L1,1.000000e-07,7.268090,0.065288,29.905572
0,0.01,0.8,L2,1.000000e-07,7.269341,0.065307,29.913571
